# PINN Implementation
This Jupyter notebook attempts to incorporate a general PINN algorithm, generally made from different sources that I have found, focussing on the development of the neural network using PyTorch.
All papers are to be accurately cited for bits of codes that I have extracted. 

## Environment Setup

In [2]:
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import deepxde as dde

torch.manual_seed(1234)
np.random.seed(1234)


Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


No backend selected.
Finding available backend...
Found pytorch
Setting the default backend to "pytorch". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


## Part I - Solving the 1D Poisson Equation
We will first solve the 1D Poisson equation through training and compare it with the FDM method and the actual solution. This part comes from the paper regarding GRINN.

In [ ]:
## We test the solution of the POISSON's EQN
#Set data type
DTYPE='float32'
torch.set_default_dtype(DTYPE)

lam = 1.0          # one wavelength
const = 1 ## we set 4 pi G  tp 1
num_of_waves = 2

rho_1 = 0.03;       # question 2b non-linear wave propagation
rho_o = 1          # zeroth order density
def pdeTest(x, y):
    dy_xx = dde.grad.hessian(y, x)
    return dy_xx - const*(rho_1 * tf.cos(2*np.pi*x/lam))

In [ ]:
## The Geometric Domain
xmin = -1
xmax = xmin + lam * num_of_waves
geom = dde.geometry.Interval(xmin, xmax)
# geom = dde.geometry.Interval(-1, 1)


## Boundaries

## Note in the deep XDE code by default for the periodic boundary u(0) = u(1)
## If the problem is periodic this condition will also satisfy u(-1) = u(1)
## Thus is most cases the left boundary is not needed. But we have the option here

def boundary_l(x, on_boundary):  # boundary x=-1 or x = 0
    if xmin == -1:
        is_on_boundary_left = on_boundary and np.isclose(x[0], xmin)
    else:
        is_on_boundary_left = on_boundary and np.isclose(x[0], 0)
    return is_on_boundary_left

def boundary_r(x, on_boundary):  # boundary x=1
    is_on_boundary_right = on_boundary and np.isclose(x[0], xmax)

    return is_on_boundary_right


bc_l = dde.icbc.PeriodicBC(geom, 0, boundary_l,derivative_order=0) ## may be used when x = [-1,1] otherwise for x =[0,1] not needed
bc_r = dde.icbc.PeriodicBC(geom, 0, boundary_r,derivative_order=0)

## 1st order terms

bc_ll = dde.icbc.PeriodicBC(geom, 0, boundary_l,derivative_order=1) ##may be used when x = [-1,1]
bc_rr = dde.icbc.PeriodicBC(geom, 0, boundary_r,derivative_order=1)

data = dde.data.TimePDE(geom,
                     pdeTest,
#                      [bc_l,bc_r,bc_ll,bc_rr], ## This is needed if one need to force periodicity on the left
                     [bc_r,bc_rr],          ## For x =[0,1] and default
                     num_domain=10000,
                     num_boundary=5000
                     #num_initial=5000,
                    # num_test=10000,
                   )

net = dde.nn.FNN([1] + [32] * 3 + [1], "sin", "Glorot normal")
model_test = dde.Model(data, net)

model_test.compile("adam", lr=1e-3)
model_test.train(iterations=3000)
model_test.compile("L-BFGS")


losshistory, train_state = model_test.train()
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

In [ ]:
X = np.linspace(xmin,xmax,1000).reshape(1000, 1)
output_0 = model_test.predict(X)
plt.plot(X,output_0,label="PINNs")
plt.ylabel(r"$\phi$")
plt.xlabel("x")
plt.legend()

Now for the classic interpretation:

In [ ]:
def fft_solver(rho,Lx,N, dim = None):

    '''
    A FFT solver that uses discrete Fast Fourier Transform to
    solve the poisson Equation:
    We apply the correction due to the finite difference grid of phi

    Input: 1. The source function: density in this case
           2. # of grid point N
           3. Domain Size in each dimension

    Output: the potential phi and the field g

    '''
    nx = N
    Lx = Lx

    dx = Lx / nx


    # Calculate the Fourier modes of the gas density
    rhohat = fft(rho)

    # Calculate the wave numbers in x and y directions
    kx = 2 * np.pi * np.fft.fftfreq(nx, dx)
    #ky = 2 * np.pi * np.fft.fftfreq(ny, dy)

    # Construct the Laplacian operator in Fourier space
    kx2 = np.meshgrid(kx**2)

#     laplace = -(kx**2 )
    ## Laplace with the correction refer to the notes
    laplace = 2*(np.cos(kx*dx)-1)

    ## Inorder to avoid the inf we replace zero with a small number
    laplace[laplace == 0] = 1e-16

    # Solve for the gravitationa potential in Fourier space
    phihat = rhohat / laplace

    phihat = rhohat * dx**2/laplace
#     window = signal.windows.tukey(len(phihat),alpha=0.25)


    # Transform back to real space to obtain the solution
#     phi = np.real(ifft(phihat*window))
    phi = np.real(ifft(phihat))

    ## The field ,i.e., gravity
    dphidx = np.gradient(phi, dx)

    return phi,dphidx

In [ ]:
from numpy.fft import fft, ifft
from scipy import signal
L = lam * num_of_waves
N = 1000
x = np.linspace(0, L, N)
rho = rho_1 * np.cos(2*np.pi*x/lam)

phi, dpidx  = fft_solver(const*(rho),L,N, dim = None)
plt.plot(x,phi,label="FFT")
plt.ylabel(r"$\phi$")
plt.xlabel("x")
plt.legend()